## Getting ready...

To illustrate how the HParams plugin works, we will use a sequential model implementation on the MNIST dataset. We'll configure HParams and compare several hyperparameter combinations in order to find the best hyperparameter optimization.

## How to do it...

1. Load in the required libraries

In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
import datetime

2. Load and prepare the MNIST dataset:

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize
x_train = x_train / 255
x_test = x_test / 255

# Set model parameters
image_width = x_train[0].shape[0]
image_height = x_train[0].shape[1]
num_channels = 1 # Greyscale = 1 channel

3. For each hyperparameter, we define the list or the interval of values to test. In this section, we'll go over three hyperparameters: the number of units per layer, the dropout rate, and the optimizer:

In [3]:
HP_ARCHITECTURE_NN = hp.HParam('archi_nn', hp.Discrete(['128, 64', '256, 128']))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.1))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

4. We now define our model to be able to test the different hyperparameter combinations:

In [9]:
def train_model(hparams, experiment_run_log_dir):
    
    nb_units = list(map(int, hparams[HP_ARCHITECTURE_NN].split(",")))
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(name='FLATTEN'))
    model.add(tf.keras.layers.Dense(units=nb_units[0], activation='relu', name='D1'))
    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT], name='DROP_OUT'))
    model.add(tf.keras.layers.Dense(units=nb_units[1], activation='relu', name='D2'))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax', name='OUTPUT'))
    
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_run_log_dir)
    hparams_callback = hp.KerasCallback(experiment_run_log_dir, hparams)
    
    model.fit(x=x_train,
              y=y_train,
              epochs=5,
              validation_data=(x_test, y_test),
              callbacks=[tensorboard_callback, hparams_callback]
             )

5. Now we iterate over the hyperparameters:

In [10]:
for archi_nn in HP_ARCHITECTURE_NN.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            hparams = {
                HP_ARCHITECTURE_NN : archi_nn,
                HP_OPTIMIZER : optimizer,
                HP_DROPOUT : dropout_rate
            }
            
            experiment_run_log_dir='logs/experiment-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            
            train_model(hparams, experiment_run_log_dir)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2351 - accuracy: 0.9315 - val_loss: 0.1292 - val_accuracy: 0.9611
Epoch 2/5
1875/1875 [==============================] - 2s 916us/step - loss: 0.0994 - accuracy: 0.9700 - val_loss: 0.0897 - val_accuracy: 0.9695
Epoch 3/5
1875/1875 [==============================] - 2s 919us/step - loss: 0.0701 - accuracy: 0.9778 - val_loss: 0.0896 - val_accuracy: 0.9729
Epoch 4/5
1875/1875 [==============================] - 2s 908us/step - loss: 0.0543 - accuracy: 0.9832 - val_loss: 0.0817 - val_accuracy: 0.9752
Epoch 5/5
1875/1875 [==============================] - 2s 922us/step - loss: 0.0424 - accuracy: 0.9865 - val_loss: 0.0921 - val_accuracy: 0.9738
Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2605 - accuracy: 0.9237 - val_loss: 0.1180 - val_accuracy: 0.9648
Epoch 2/5
1875/1875 [==============================] - 2s 942us/step - loss: 0.1186 - accuracy: 0.9638 - val_loss: 0.1000 - val_accura

6. And we'll start the TensorBoard application...

In [12]:
%load_ext tensorboard
%tensorboard --logdir='logs'

The above interactive application lets us visualize and review the performance of our different models based on the hyperparameter sweep. Going to 'HPARAMS' -> 'TABLE VIEW' demonstrates the best performance was achieved with 0.1 dropout, 256, 128 nodes, and adam optimizer. The 'PARALLEL COORDINATES VIEW' enables a great visual comparison and quick determination of best performance.